In [3]:
import os
import json

result_dir = "../exp_results"
num_cores = [1]
num_tasks = [3,6,9,12]
utilizations = [0.4, 0.6, 0.8]
num_tasksets = 30
summary_path = result_dir + "/summary.csv"

# if summary file exists, remove it
if os.path.exists(summary_path):
    os.remove(summary_path)
    
header = "numCores,numTasks,utilization,tasksetIndex,realLinux_schedulability"
with open(summary_path, "a") as f:
    f.write(header + "\n")

for num_core in num_cores:
    for num_task in num_tasks:
        for utilization in utilizations:
            for i in range(num_tasksets):
                result_file = result_dir + "/details/" + str(num_core) + "cores/" + str(num_task) + "tasks/" + str(utilization) + "utilization/result_" + str(i) + ".json"
                with open(result_file, 'r') as file:
                    tasks_result = json.load(file)
                assert len(tasks_result) == num_task
                
                realLinux_schedulability = True
                for task_result in tasks_result:
                    if task_result["wcrt_ns"] > task_result["deadline_ns"]:
                        realLinux_schedulability = False
                        break
                with open(summary_path, "a") as f:
                    f.write(str(num_core) + "," + str(num_task) + "," + str(utilization) + "," + str(i) + "," + str(realLinux_schedulability) + "\n")
                


In [6]:
import json
import pandas as pd

# analyze the exp with non_rt_task
result_dir = "../exp_results_non_RT"
schedulers = ["CFS", "FIFO", "RR", "RM"]

# set task_info in dataset
result_path = result_dir + "/" + schedulers[-1] + "_result.json"
results_df = pd.read_json(result_path)
results_df = results_df[["task_name", "deadline_ns", "wcet_ns", "priority"]]

for scheduler in schedulers:
    result_path = result_dir + "/" + scheduler + "_result.json"
    result_df = pd.read_json(result_path)
    result_df = result_df[["task_name", "wcrt_ns"]]
    result_df.columns = ["task_name", scheduler + "_wcrt_ns"]
    results_df = pd.merge(results_df, result_df, on="task_name")

# convert to ms
results_df["deadline_ms"] = results_df["deadline_ns"] / 1000000
results_df["wcet_ms"] = results_df["wcet_ns"] / 1000000
results_df["CFS_wcrt_ms"] = results_df["CFS_wcrt_ns"] / 1000000
results_df["FIFO_wcrt_ms"] = results_df["FIFO_wcrt_ns"] / 1000000
results_df["RR_wcrt_ms"] = results_df["RR_wcrt_ns"] / 1000000
results_df["RM_wcrt_ms"] = results_df["RM_wcrt_ns"] / 1000000

# round to 2 decimal places
results_df = results_df.round(1)

# drop ns columns
results_df = results_df.drop(["deadline_ns", "wcet_ns", "CFS_wcrt_ns", "FIFO_wcrt_ns", "RR_wcrt_ns", "RM_wcrt_ns"], axis=1)
results_df

,task_name,priority,deadline_ms,wcet_ms,CFS_wcrt_ms,FIFO_wcrt_ms,RR_wcrt_ms,RM_wcrt_ms
0,task1,84,100.0,2.8,14.8,855.5,334.8,6.2
1,task2,74,3000.0,723.0,1882.9,723.0,1144.8,1265.9
2,task3,88,50.0,1.8,14.4,723.2,223.2,1.9
3,task4,81,200.0,27.4,42.7,605.5,322.5,33.6
4,task5,84,100.0,1.4,14.7,700.0,240.0,6.1
5,task6,77,500.0,82.7,186.1,505.5,303.8,124.2
6,Non_RT_Task,0,500.0,100.1,372.2,1005.5,1029.7,1000.5
